In [1]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
caltech_dir = "./data/image/101_ObjectCategories"
categories = ["chair","camera","butterfly","elephant","flamingo"]
nb_classes = len(categories)

#### 이미지 크기 지정

In [4]:
image_w = 64
image_h = 64
pixels = image_w * image_h*3 #64x64 픽셀 x3(RGB)

#### 이미지 데이터 읽기

In [16]:
X=[]
Y=[]

for idx, cat in enumerate(categories) :
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    #print(label, idx, cat) ex) [1,0,0,0,0] chair
    
    #이미지
    image_dir = caltech_dir+"/"+cat
    files = glob.glob(image_dir+"/*.jpg") #확장자가 jpg인 파일 모두
    
    for i, f in enumerate(files) :
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        
        data = np.asarray(img)
        
        X.append(data)
        Y.append(label)
            
X = np.array(X)
Y = np.array(Y)

#### train test data나누기

In [21]:
X_train, X_test, y_train,y_test = train_test_split(X,Y)

#### CNN으로 분류

In [19]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [42]:
#데이터 정규화
X_train = X_train.astype('float')/256 #250개
X_test = X_test.astype('float')/256 #84개
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (250, 64, 64, 3)
X_test shape: (84, 64, 64, 3)


##### modeling

In [38]:
model = Sequential()
model.add(Convolution2D(filters = 32,kernel_size=(3,3),  padding = 'same',input_shape = X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64,(3,3)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

#### training

In [37]:
model.fit(X_train, y_train, batch_size = 32, epochs=50)

Epoch 1/50
250/250 [==============================] - 10s 38ms/step - loss: 1.1129 - acc: 0.7544
Epoch 2/50
250/250 [==============================] - 7s 26ms/step - loss: 0.5218 - acc: 0.7880
Epoch 3/50
250/250 [==============================] - 7s 26ms/step - loss: 0.4790 - acc: 0.7992
Epoch 4/50
250/250 [==============================] - 6s 26ms/step - loss: 0.4404 - acc: 0.8208
Epoch 5/50
250/250 [==============================] - 6s 26ms/step - loss: 0.3982 - acc: 0.8208
Epoch 6/50
250/250 [==============================] - 7s 26ms/step - loss: 0.3586 - acc: 0.8448
Epoch 7/50
250/250 [==============================] - 6s 26ms/step - loss: 0.2866 - acc: 0.8864
Epoch 8/50
250/250 [==============================] - 7s 26ms/step - loss: 0.2466 - acc: 0.9040
Epoch 9/50
250/250 [==============================] - 6s 26ms/step - loss: 0.2046 - acc: 0.9144
Epoch 10/50
250/250 [==============================] - 6s 26ms/step - loss: 0.1354 - acc: 0.9424
Epoch 11/50
250/250 [=================

#### model test

In [39]:
score = model.evaluate(X_test, y_test)
print('loss', score[0])
print('accuracy', score[1])

84/84 [==============================] - 1s 14ms/step
loss 0.5052914718786875
accuracy 0.800000011920929


In [44]:
pre = model.predict(X_test)
pre[:10]

array([[0.20021439, 0.19997953, 0.19992296, 0.20002036, 0.19986278],
       [0.20020321, 0.20002335, 0.19988987, 0.1999705 , 0.1999131 ],
       [0.20021214, 0.20002402, 0.19989094, 0.19998041, 0.19989246],
       [0.20029922, 0.19999154, 0.19992146, 0.19996373, 0.19982402],
       [0.20015404, 0.20004293, 0.19996516, 0.19993195, 0.19990586],
       [0.20022047, 0.19998151, 0.19995967, 0.19991297, 0.1999255 ],
       [0.20027465, 0.1999446 , 0.19995582, 0.19996391, 0.19986098],
       [0.20023803, 0.19992854, 0.19993496, 0.20002617, 0.19987231],
       [0.20019175, 0.19998994, 0.19990517, 0.20002583, 0.19988728],
       [0.2002375 , 0.19996822, 0.20000176, 0.19996297, 0.19982953]],
      dtype=float32)

In [45]:
pre.shape

(84, 5)

In [59]:
for i,v in enumerate(pre) :
    pred = v.argmax() #예측한 값
    ans = y_test[i].argmax() #정답
    dat = X_test[i] #이미지데이터
    
    #예측값과 정답이 같으면 그냥 지나갈 것
    if ans ==pred :continue 
        
    print("[NG]", categories[pred], "!=", categories[ans])
    print(v)

[NG] chair != butterfly
[0.20021439 0.19997953 0.19992296 0.20002036 0.19986278]
[NG] chair != flamingo
[0.20020321 0.20002335 0.19988987 0.1999705  0.1999131 ]
[NG] chair != elephant
[0.20021214 0.20002402 0.19989094 0.19998041 0.19989246]
[NG] chair != camera
[0.20029922 0.19999154 0.19992146 0.19996373 0.19982402]
[NG] chair != flamingo
[0.20015404 0.20004293 0.19996516 0.19993195 0.19990586]
[NG] chair != flamingo
[0.20022047 0.19998151 0.19995967 0.19991297 0.1999255 ]
[NG] chair != elephant
[0.20019175 0.19998994 0.19990517 0.20002583 0.19988728]
[NG] chair != butterfly
[0.2002375  0.19996822 0.20000176 0.19996297 0.19982953]
[NG] chair != elephant
[0.20016912 0.2000547  0.19991282 0.19997156 0.19989182]
[NG] chair != flamingo
[0.20029204 0.19999243 0.19991404 0.20000957 0.19979195]
[NG] chair != camera
[0.20025784 0.20000619 0.19989909 0.20000196 0.19983494]
[NG] chair != butterfly
[0.20021549 0.20000152 0.1999387  0.19994338 0.19990082]
[NG] chair != flamingo
[0.20025474 0.1999